In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import pymc as pm
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm

from tbdynamics.utils import round_sigfig
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
from summer2.functions.time import get_sigmoidal_interpolation_function
from summer2 import AgeStratification, Stratification, Overwrite, Multiply
from tbdynamics.inputs import fixed_parameters

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Defining initial variables

In [3]:
pd.options.plotting.backend = 'plotly'
time_start = 1800
time_end = 2020
time_step = 0.1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50,70]

## Load data

In [4]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'

### Process Birth data

In [5]:
birth_rates = pd.read_csv(Path(DATA_PATH / 'vn_birth.csv'), index_col=0)['value']

### Process Death Data

In [6]:
# Path to the CSV file
file_path = './data/data.csv' #from UN Population

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path, usecols=['Age', 'Time', 'Population', 'Deaths'])
data = data.set_index(['Age', 'Time'])
data.index = data.index.swaplevel()
age_groups = set(data.index.get_level_values(1))
years = set(data.index.get_level_values(0))

In [7]:
def get_age_groups_in_range(age_groups, lower_limit, upper_limit):
    return [i for i in age_groups if '+' not in i and lower_limit <= int(i.split('-')[0]) <= upper_limit]

In [8]:
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 69], [70, 200]]
agegroup_map = {low: get_age_groups_in_range(age_groups, low, up) for low, up in agegroup_request}
agegroup_map[agegroup_request[-1][0]].append('100+')
mapped_rates = pd.DataFrame()
for year in years:
    for agegroup in agegroup_map:
        age_mask = [i in agegroup_map[agegroup] for i in data.index.get_level_values(1)]
        age_year_data = data.loc[age_mask].loc[year, :]
        total = age_year_data.sum()
        mapped_rates.loc[year, agegroup] = total['Deaths'] / total['Population']
mapped_rates.index = mapped_rates.index + 0.5
death_df = mapped_rates.loc[birth_rates.index]

## Creating model

In [9]:
tb_model = CompartmentalModel(
    times=(time_start, time_end),
    compartments=compartments,
    infectious_compartments=infectious_compartments,
    timestep=time_step,
)

In [10]:
## Set init pop
start_pop = Parameter("start_population_size")
init_pop = {
        "infectious": 0,
        "susceptible": start_pop - 0,
}
tb_model.set_initial_population(init_pop)

In [11]:
## Add birth process
process = "birth"
destination = "susceptible"
crude_birth_rate = get_sigmoidal_interpolation_function(
    birth_rates.index, birth_rates
)
tb_model.add_crude_birth_flow(
        process,
        crude_birth_rate,
        destination,
)

In [12]:
## Adding death process
process = "universal_death" ## will be adjusted later
universal_death_rate = 1.0
tb_model.add_universal_death_flows("universal_death", death_rate=universal_death_rate)

In [13]:
## Stratifying by age
age_strat = AgeStratification("age", age_strata, compartments)
universal_death_funcs, death_adjs = {}, {}
for age in age_strata:
    universal_death_funcs[age] = get_sigmoidal_interpolation_function(
        death_df.index, death_df[age]
    )
    death_adjs[str(age)] = Overwrite(universal_death_funcs[age])
age_strat.set_flow_adjustments("universal_death", death_adjs)
tb_model.stratify_with(age_strat)

In [14]:
def get_gender_strat(age_strata, compartments, fixed_params):
    requested_strata = fixed_params["gender"]["strata"]
    strat = Stratification("gender", requested_strata, compartments)
    # Pre-process generic flow adjustments:
    # IF infection is adjusted and other infection flows NOT adjusted
    # THEN use the infection adjustment for the other infection flows

    adjs = {}
    # Set birth flow adjustments
    adjs["birth"] = fixed_params["gender"]["proportions"]
    # # # Set birth flow adjustments. 
    for flow_name, adjustment in adjs.items():
        if flow_name == 'birth':
            adj = {k : Multiply(v) for k,v in adjustment.items()} 
            strat.set_flow_adjustments(flow_name, adj)
    return strat

In [15]:
gender_strat = get_gender_strat(age_strata, compartments, fixed_parameters)

In [16]:
tb_model.stratify_with(gender_strat)

In [17]:
tb_model.flows

[<CrudeBirthFlow 'birth' to susceptibleXage_0Xgender_male>,
 <CrudeBirthFlow 'birth' to susceptibleXage_0Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_0Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_0Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_5Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_5Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_15Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_15Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_35Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_35Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_50Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_50Xgender_female>,
 <DeathFlow 'universal_death' from susceptibleXage_70Xgender_male>,
 <DeathFlow 'universal_death' from susceptibleXage_70Xgender_female>,
 <DeathFlow 'universal_death' from early_latentXage_0X

In [18]:
def request_compartment_output(model, output_name, genders, compartments, save_results=True):
        model.request_output_for_compartments(
            output_name, compartments, save_results=save_results
        )
        for gender_stratum in genders:
            # For location-specific mortality calculations
            gender_output_name = f"{output_name}Xgender_{gender_stratum}"
            model.request_output_for_compartments(
                gender_output_name,
                compartments,
                strata={"gender": gender_stratum},
                save_results=save_results,
            )

In [19]:
def request_compartment_output(model, output_name, ages,genders,compartments, save_results=True):
    model.request_output_for_compartments(
            output_name, compartments, save_results=save_results
    )
    for age_stratum in ages:
        # For age-specific population calculations
        age_output_name = f"{output_name}Xage_{age_stratum}"
        for gender_stratum in genders:
            age_genders_output_name = f"{age_output_name}Xgender_{gender_stratum}"
            model.request_output_for_compartments(
                age_genders_output_name,
                compartments,
                strata={'age': str(age_stratum)}, # I've added str to age stratum, It worked
                save_results=save_results,
            )

In [20]:
request_compartment_output(tb_model, "total_population", age_strata, fixed_parameters["gender"]["strata"],compartments)

## Optimizing model

In [21]:
params = {
    "start_population_size": 12400000,
}
priors = [
    esp.UniformPrior("start_population_size", (1000000, 20000000)),
]
pop = pd.Series({2009: 86025000, 2019: 96484000})


targets = [
    est.NormalTarget('total_population', pop, stdev= 1000000),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [22]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')

map_params


Best calibration parameters found:
   start_population_size: 1069000.0 (within bound (1000000.0, 20000000.0)


{'start_population_size': 1069495.628981531}

In [23]:
params.update(map_params)
tb_model.run(params)
params
derived_df_0 = tb_model.get_derived_outputs_df()

In [24]:
derived_df_0.columns

Index(['total_population', 'total_populationXage_0Xgender_female',
       'total_populationXage_0Xgender_male',
       'total_populationXage_15Xgender_female',
       'total_populationXage_15Xgender_male',
       'total_populationXage_35Xgender_female',
       'total_populationXage_35Xgender_male',
       'total_populationXage_50Xgender_female',
       'total_populationXage_50Xgender_male',
       'total_populationXage_5Xgender_female',
       'total_populationXage_5Xgender_male',
       'total_populationXage_70Xgender_female',
       'total_populationXage_70Xgender_male'],
      dtype='object')

In [25]:
pattern = 'total_populationXage_'
pop = derived_df_0[[col for col in derived_df_0.columns if col.startswith(pattern)]] 

In [26]:
pop.to_csv('data/outpop.csv')

In [27]:
pop.plot.area()

In [28]:
pop.plot.area(log_y=True)

In [29]:
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [86025000, 96484000],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },  
}

In [30]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3

In [31]:
derived_df_0.index

Float64Index([1800.0, 1800.1, 1800.2, 1800.3, 1800.4, 1800.5, 1800.6, 1800.7,
              1800.8, 1800.9,
              ...
              2019.1, 2019.2, 2019.3, 2019.4, 2019.5, 2019.6, 2019.7, 2019.8,
              2019.9, 2020.0],
             dtype='float64', length=2201)

In [32]:
def plot_population_pyramid_plotly(year, df):
    """
    Plots the population pyramid for a given year using Plotly.

    :param year: Year for which the population pyramid is to be plotted.
    :param df: DataFrame containing the population data.
    """

    # Find the row in the DataFrame that is closest to the requested year
    closest_year_data = df.iloc[year]

    # Define the age groups
    age_groups = ['0', '5', '15', '35', '50', '70']

    # Extracting population data for the closest year
    population_female = [-closest_year_data[f'total_populationXage_{age}Xgender_female'] for age in age_groups]
    population_male = [closest_year_data[f'total_populationXage_{age}Xgender_male'] for age in age_groups]

    # Create the figure
    fig = go.Figure()

    # Add traces for male and female populations
    fig.add_trace(go.Bar(
        y=age_groups,
        x=population_male,
        name='Male',
        orientation='h',
        marker=dict(color='blue')
    ))

    fig.add_trace(go.Bar(
        y=age_groups,
        x=population_female,
        name='Female',
        orientation='h',
        marker=dict(color='red')
    ))

    # Update layout
    fig.update_layout(
        title=f'Population Pyramid for the Year {year}',
        xaxis_title="Population",
        yaxis_title="Age Group",
        barmode='relative',
        bargap=0.2
    )

    # Show the figure
    fig.show()

# Example usage of the function
plot_population_pyramid_plotly(2000, pop)